In [1]:
# Load env + connect to SQL Server

In [2]:
import os
from dotenv import load_dotenv
import pyodbc
import pandas as pd

# Load environment variables
load_dotenv()

server = os.getenv("MSSQL_HOST")
database = os.getenv("MSSQL_DB")
username = os.getenv("MSSQL_USER")
password = os.getenv("MSSQL_PASS")

connection_string = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
)

conn = pyodbc.connect(connection_string)
print("✅ Good to go!")

✅ Good to go!


In [3]:
# Helper Function
def query(sql):
    df = pd.read_sql(sql, conn)
    return df

In [4]:
# Simple test
query("SELECT TOP 5 * FROM automations.people;")

/tmp/ipykernel_53046/498837797.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,id,companyID,sponsorID,fullName,title,email,phone,linkedin,leadSource,created_at,updated_at
0,23,47,None,Viki Tyrer,None,vtyrer0@opera.com,55(147)524-2845,None,Mockaroo,2025-11-18 10:24:04.084066,2025-11-18 10:24:04.084066
1,24,59,None,Drake Menel,None,dmenel1@skype.com,44(932)419-7234,None,Mockaroo,2025-11-18 10:24:04.318441,2025-11-18 10:24:04.318441
2,25,56,None,Shell Wishkar,None,swishkar2@addthis.com,358(997)500-3078,None,Mockaroo,2025-11-18 10:24:04.537192,2025-11-18 10:24:04.537192
3,26,70,None,Melva Rizzetti,None,mrizzetti3@spiegel.de,1(298)364-3454,None,Mockaroo,2025-11-18 10:24:04.755934,2025-11-18 10:24:04.755934
4,27,55,None,Kikelia Binham,None,kbinham4@gravatar.com,46(453)947-3858,None,Mockaroo,2025-11-18 10:24:04.974693,2025-11-18 10:24:04.974693


In [5]:
========= SCENARIOS ==========

SyntaxError: invalid syntax (1732213722.py, line 1)

In [ ]:
# Count people by company
query("""
SELECT c.name AS company_name, COUNT(p.id) AS total_people
FROM automations.people p
JOIN automations.companies c ON p.companyID = c.id
GROUP BY c.name
ORDER BY total_people DESC;
""")


In [ ]:
# People who purchased products
query("""
SELECT 
    p.fullName,
    c.name AS company,
    pr.title AS product,
    cp.value,
    cp.purchaseDate
FROM automations.companyProducts cp
JOIN automations.people p ON cp.personID = p.id
JOIN automations.products pr ON cp.productID = pr.id
JOIN automations.companies c ON cp.companyID = c.id
ORDER BY cp.purchaseDate DESC;
""")


In [ ]:
# Supplier network graph
query("""
SELECT 
    c.name AS company,
    s.name AS supplier,
    pr.title AS product
FROM automations.suppliers sp
JOIN automations.companies c ON sp.companyID = c.id
JOIN automations.companies s ON sp.supplierID = s.id
JOIN automations.products pr ON sp.productID = pr.id
ORDER BY c.name;
""")


In [ ]:
# Company KPIs
query("""
SELECT 
    c.name,
    COUNT(DISTINCT p.id) AS employees,
    COUNT(DISTINCT cp.productID) AS purchased_products,
    SUM(cp.value) AS total_spend
FROM automations.companies c
LEFT JOIN automations.people p ON p.companyID = c.id
LEFT JOIN automations.companyProducts cp ON cp.companyID = c.id
GROUP BY c.name
ORDER BY total_spend DESC;
""")


In [ ]:
# Product Performance
query("""
SELECT 
    pr.title,
    pr.category,
    COUNT(cp.id) AS total_sales,
    SUM(cp.value) AS revenue
FROM automations.products pr
LEFT JOIN automations.companyProducts cp ON cp.productID = pr.id
GROUP BY pr.title, pr.category
ORDER BY revenue DESC;
""")


In [ ]:
======== VISUALS ========

In [6]:
import os
import urllib
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

# -----------------------------
# SQL Connection (with escaping)
# -----------------------------
server = os.getenv("MSSQL_HOST")
database = os.getenv("MSSQL_DB")
username = os.getenv("MSSQL_USER")
password = os.getenv("MSSQL_PASS")

params = urllib.parse.quote_plus(
    f"Driver=ODBC Driver 18 for SQL Server;"
    f"Server={server};"
    f"Database={database};"
    f"UID={username};"
    f"PWD={password};"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# -----------------------------
# Query Helper
# -----------------------------
def query(sql):
    return pd.read_sql(sql, engine)

# -----------------------------
# Plot Helper
# -----------------------------
def show_bar(df, x, y, title="", xlabel="", ylabel="", rot=0):
    plt.figure(figsize=(12,6))
    plt.bar(df[x], df[y])
    plt.title(title)
    plt.xlabel(xlabel or x)
    plt.ylabel(ylabel or y)
    plt.xticks(rotation=rot)
    plt.show()

def show_line(df, x, y, title="", xlabel="", ylabel="", rot=0):
    plt.figure(figsize=(12,6))
    plt.plot(df[x], df[y])
    plt.title(title)
    plt.xlabel(xlabel or x)
    plt.ylabel(ylabel or y)
    plt.xticks(rotation=rot)
    plt.show()


In [7]:
# People count per company
df = query("""
SELECT companyID AS company_id, COUNT(*) AS total_people
FROM automations.people
GROUP BY companyID
ORDER BY total_people DESC;
""")

df.head()
show_bar(df, "company_id", "total_people",
         title="People Count per Company",
         xlabel="Company ID", ylabel="People Count")


InterfaceError: (pyodbc.InterfaceError) ('28000', "[28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed for user 'ncz_automation'. (18456) (SQLDriverConnect)")
(Background on this error at: https://sqlalche.me/e/20/rvf5)